In [ ]:
import torch
from torch import utils
from torchvision import datasets, transforms
import matplotlib
import matplotlib.pyplot as plt
from torch.autograd import Variable
import sys
sys.path.append('../')
import orch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np

%matplotlib inline

In [ ]:
def string_to_clicked(s):
    step = 0
    clicked = np.zeros((16, 16), dtype = np.int8)
    game = s.split()
    
    if game[0] == 'white':
        winner = 1
    elif game == 'unknown':
        winner = 0
    else: 
        winner = -1
        
    for move in game[1:]:
        if move[0] > 'p' or move[0] < 'a':
            winner = 0
            break
            
        cell = -1
        if step % 2 == 0:
            cell = 1
        x = ord(move[0]) - ord('a') - 1
        if (move[0] > 'i'):
            x -= 1
        y = int(move[1:]) - 1
        clicked[ord(move[0]) - ord('a') - 1][int(move[1:]) - 1] = cell
        step += 1
        
    return winner, clicked

In [ ]:
def make_data(game, data, data_answers):
    step = 0
    clicked = np.zeros((16, 16), dtype = np.int8)
    
    if game[0] == 'white':
        winner = 1
    elif game[0] == 'black':
        winner = -1
    else:
        winner = 0
    desk = np.zeros((2, 16, 16), dtype = np.int8)

    desk[0] = clicked
    desk[1] = np.ones((16, 16), dtype = np.int8) * winner
    data = np.vstack((data, desk.reshape(1, 2, 16, 16)))
    for move in game[1:- 1]:
        if winner == 0:
            contunue
        cell = -1
        if step % 2 == 0:
            cell = 1
        clicked[ord(move[0]) - ord('a') - 1][int(move[1:]) - 1] = cell
        if cell == winner:
            desk = np.zeros((2, 16, 16), dtype = np.int8)
            desk[0] = clicked
            desk[1] = np.ones((16, 16), dtype = np.int8) * winner
            data = np.vstack((data, desk.reshape(1, 2, 16, 16)))
            data_answers = np.vstack((data_answers, np.array([ord(move[0]) - ord('a') - 1, int(move[1:]) - 1])))
        step += 1
    move = game[len(game) - 1]
    data_answers = np.vstack((data_answers, np.array([ord(move[0]) - ord('a') - 1, int(move[1:]) - 1])))
    
    return data, data_answers

In [ ]:
def make_batch(data, data_answers, batch_sz, i):
    with open("../input/ggaammeess/text.txt") as f:
        j = 0
        num_desk = 0

        for line in f:
            if j < i:
                continue
                
            if batch_sz < num_desk:
                print(num_desk)
                break

            if i == 0:

                game = line.split()
                step = 0
                clicked = np.zeros((16, 16), dtype = np.int8)

                if game[0] == 'white':
                    winner = 1
                elif game[0] == 'black':
                    winner = -1
                else: 
                    continue

                desk = np.zeros((2, 16, 16), dtype = np.int8)
                desk[0] = clicked
                desk[1] = np.ones((16, 16), dtype = np.int8) * winner
                data[0] = desk

                num_desk += 1
                for move in game[1:- 1]:
                    cell = -1
                    if step % 2 == 0:
                        cell = 1
                    clicked[ord(move[0]) - ord('a') - 1][int(move[1:]) - 1] = cell
                    if cell == winner:
                        desk = np.zeros((2, 16, 16), dtype = np.int8)
                        desk[0] = clicked
                        desk[1] = np.ones((16, 16), dtype = np.int8) * winner

                        data = np.vstack((data, desk.reshape(1, 2, 16, 16)))
                        x = np.array(ord(move[0]) - ord('a') - 1)
                        y = int(move[1:]) - 1
                        pos = np.array([x, y])
                        data_answers = np.vstack((data_answers, pos))
                        num_desk += 1

                    step += 1
                move = game[len(game) - 1]
                data_answers = np.vstack((data_answers, np.array([ord(move[0]) - ord('a') - 1, int(move[1:]) - 1])))
                data_answers = data_answers[1:]

            else:
                game = line.split()
                data, data_answers = make_data(game, data, data_answers)

            j += 1
            
    return data, data_answers, j        


In [ ]:
data = np.zeros((1, 2, 16, 16), dtype = np.int8)
data_answers = np.zeros((1, 2), dtype = np.uint8)

batch_sz = 50
i = 0 #количество прочитанных строк

data, data_answers, i = make_batch(data, data_answers, batch_sz, i)
data = data[-batch_sz:]
data_answers = data_answers[-batch_sz:]

In [ ]:
print(data.shape, data_answers.shape, i)

data_sz = data.shape[0]
test_sz = data_sz // 5
train_sz = data_sz - test_sz
x_train, x_test, y_train, y_test = train_test_split(data, data_answers, test_size = test_sz)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


In [ ]:
#переконвертим пару в вектор
def sell_to_int(cell):
    return (15 * cell[0] + cell[1] + 1)

y_train = y_train.astype(np.long)
y_test = y_test.astype(np.long)
ttrain = torch.zeros(train_sz, dtype = torch.long)
ttest = torch.zeros(test_sz, dtype = torch.long)
new_y_train = np.zeros((train_sz), dtype = np.long)
new_y_test = np.zeros((test_sz), dtype = np.long)
for i in range(train_sz):
    new_y_train[i] = sell_to_int(y_train[i])
print(new_y_train.shape, ttrain.size())
ttrain = torch.from_numpy(new_y_train)
    
for i in range(test_sz):
    new_y_test[i] = sell_to_int(y_test[i])
ttest = torch.from_numpy(new_y_test)

print(ttrain)

x_train = x_train.astype(np.int32)
x_test = x_test.astype(np.int32)
x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
print(x_train.size(), x_train[0].size())


In [ ]:
class CNN(torch.nn.Module):    
    def __init__(self):
        super(CNN, self).__init__()
        #2 * 16 * 16
        self.conv1 = torch.nn.Conv2d(2, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        # 16 * 16 * 16
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 16 x 8 x 8
        self.conv3 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # 32 x 8 x 8
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 64 * 8 * 8
        self.conv5 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        # 128 * 8 * 8
        #pool
        # 128 x 4 x 4
        self.conv6 = torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        # 256 x 4 x 4
        self.fc1 = torch.nn.Linear(256 * 4 * 4, 1024)
        self.fc2 = torch.nn.Linear(1024, 255)
        
    def forward(self, x):
        x = x.view(-1, 2, 16, 16)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        x = F.relu(self.conv6(x))
        x = x.view(-1, 256 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return(x)
    
model = CNN()

In [ ]:
l1_lmbd = 0.15
l2_lmbd = 0.25
def li_loss(layer, i):
    return torch.norm(layer.weight.data, p=i)

In [ ]:
def data_train_to_torch(data, data_answers, batch_sz):
    train_sz = batch_sz
    data_sz = data.shape[0]
    test_sz = 0
    train_sz = data_sz - test_sz
    x_train, x_test, y_train, y_test = train_test_split(data, data_answers, test_size = test_sz)
    
    y_train = y_train.astype(np.long)
    ttrain = torch.zeros(train_sz, dtype = torch.long)
    new_y_train = np.zeros((train_sz), dtype = np.long)
    for i in range(train_sz):
        new_y_train[i] = sell_to_int(y_train[i])
    ttrain = torch.from_numpy(new_y_train)

    x_train = x_train.astype(np.float64)
    x_train = torch.DoubleTensor(x_train)
    
    return ttrain, x_train

In [ ]:

def train(model, epoch, data, data_answers, batch_sz, i):
    loss = 0
    now = 0
    
    #будем формировать батчи на ходу, как и выбирать train_sz
    train_sz = 300000
    for j in range(train_sz // batch_sz):
        #формируем не просто трейн, а сразу батч
        data, data_answers, i = make_batch(data, data_answers, batch_sz, i)
        i += 1 #начинаем для следующей строки
        data = data[-batch_sz:]
        data_answers = data_answers[-batch_sz:]
        tdata, data_b = data_train_to_torch(data, data_answers, batch_sz)
        
        optimizer.zero_grad()
        data_b = data_b.cuda().float()
        tdata = tdata.cuda().long()
        output = model(data_b)
        loss = F.cross_entropy(output, tdata)
        #print(loss.data.item())
        loss += l1_lmbd * li_loss(model.fc1, 1) \
        +  l1_lmbd * li_loss(model.fc2, 1) +  l2_lmbd * li_loss(model.fc1, 2) +  l2_lmbd * li_loss(model.fc2, 2)
        #loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        now += batch_sz
        if j % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, now, train_sz,
                100. * now / train_sz, loss.data.item()))


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
model.load_state_dict(torch.load("../input/model-9/model3 (1)"))
model.eval()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
num_epoch = 10
for epoch in range(1, num_epoch):
    i = 0
    train(model, epoch, data, data_answers, batch_sz, i)
    print("epochs: ", epoch)


In [ ]:
torch.save(model.state_dict(), "./model3-2")

In [8]:
import torch
from torch import utils
from torchvision import datasets, transforms
import matplotlib
import matplotlib.pyplot as plt
from torch.autograd import Variable
import sys
sys.path.append('../')
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np

%matplotlib inline


class CNN(torch.nn.Module):    
    def __init__(self):
        super(CNN, self).__init__()
        #2 * 16 * 16
        self.conv1 = torch.nn.Conv2d(2, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        # 16 * 16 * 16
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 16 x 8 x 8
        self.conv3 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # 32 x 8 x 8
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 64 * 8 * 8
        self.conv5 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        # 128 * 8 * 8
        #pool
        # 128 x 4 x 4
        self.conv6 = torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        # 256 x 4 x 4
        self.fc1 = torch.nn.Linear(256 * 4 * 4, 1024)
        self.fc2 = torch.nn.Linear(1024, 255)
        
    def forward(self, x):
        x = x.view(-1, 2, 16, 16)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        x = F.relu(self.conv6(x))
        x = x.view(-1, 256 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return(x)


def model_policy(clicked, step, model, not_used):
    if step % 2 == 0:
        walk = 1
    else:
        walk = -1
    i, j = gm_has_four_line(walk, clicked)
    if i != -1 and j != -1:
        print(i + 1, j + 1, 'FIND')
        return i * 15 + j
    
    #сделаем тензор
    data = np.zeros((2, 16, 16))
    for i in range(15):
        for j in range(15):
            data[0][i][j] = clicked[i][j]
    data[1] = np.ones((16, 16)) * walk
    data = torch.from_numpy(data)
    data = data.float()
    ans = model(data)

    max_ind = torch.max(ans, 1)
    max_ind_1 = max_ind[1].numpy()
    
    while((max_ind_1[0] + 1) not in not_used):
        ans[0][max_ind_1[0]] = ans[0][max_ind_1[0]] - 100
        max_ind = torch.max(ans, 1)
        max_ind_1 = max_ind[1].numpy()
    
    return max_ind_1[0]


def string_to_clicked(s):
    step = 0
    clicked = np.zeros((15, 15), dtype = np.int8)
    game = s.split()
    gamer = 1


    for move in game:
        if move[0] > 'p' or move[0] < 'a':
            break
            
        cell = -1clicked = np.zeros((15, 15), dtype = np.int8)  # Создаем сет для клеточек, по которым мы кликнули
not_used = list(range(1, 15*15 + 1))
        if (step % 2 == 0):
            cell = 1
        clicked[ord(move[0]) - ord('a') - 1][int(move[1:]) - 1] = cell
        step += 1
        
    if step % 2 == 0:
    	step = 1
    else:
    	step = -1


    return step, clicked


def gm_has_four_line(gamer, clicked):
    w = 0
    for i in range(15):     #по строкам
        w = 0
        for j in range(15):
            if clicked[i][j] == gamer:
                w += 1
            else:
                w = 0
            if w >= 4:
                if j < 14 and clicked[i][j + 1] == 0:
                    return i, j + 1
                elif j - 4 >= 0 and clicked[i][j - 4] == 0:
                    return i, j - 4
            
    for j in range(15):          #по столбцам
        w = 0
        for i in range(15):
            if clicked[i][j] == gamer:
                w += 1
            else:
                w = 0
            if w >= 4:
                if i < 14 and clicked[i + 1][j] == 0:
                    return i + 1, j
                elif i - 4 >= 0 and clicked[i - 4][j] == 0:
                    return i - 4, j
           
    for i in range(12):
        w = 0                                      #по диагонали нижняя половина
        for k in range(15 - i):
            if clicked[i + k][k] == gamer:
                w += 1
            else:
                w = 0

            if w >= 4:
                if i + k + 1 <= 14 and k + 1 <= 14 and clicked[i + 1 + k][k + 1] == 0:
                    return i + 1 + k, k + 1
                elif i + k - 4 >= 0 and k - 4 >= 0 and clicked[i + k - 4][k - 4] == 0:
                    return i + k - 4, k - 4
       
    for i in range(12):
        w = 0                                      #по диагонали верхная половина
        for k in range(15 - i):
            if clicked[k][i + k] == gamer:
                w += 1
            else:
                w = 0
            if w >= 4:
                if i + k + 1 <= 14 and k + 1 <= 14 and clicked[1 + k][k + i + 1] == 0:
                    return 1 + k, k + 1 + i
                elif i + k - 4 >= 0 and k - 4 >= 0 and clicked[k - 4][k + i - 4] == 0:
                    return k - 4, k - 4 + i
                       
            
    return -1, -1

def to_out(cell):
	x = cell % 15
	y = cell // 15
	out = ord('a') + x
	ans = chr(out) + str(y) + '\n'
	return ans


 def run_model(model, log):
 	clicked = np.zeros((15, 15), dtype = np.int8)  # Создаем сет для клеточек, по которым мы кликнули
	not_used = list(range(1, 15*15 + 1))
    step, clicked, not_used = str_to_clicked(log)
	not_used = list(range(1, 15*15 + 1))

    for i in range(1, 256):
		if clicked[(i - 1) % 15][(i - 1) // 15] != 0:
			not_used.remove(i)

    cell = model_policy(clicked, step, model, not_used)

    return(to_out(cell))


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

model = CNN()
model.load_state_dict(torch.load("/home/nata/model_rand_3", map_location='cpu'))
model.eval()

log = input()

ans = run_model(model, log)

print(ans)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 106)